In [1]:
import scikinC
from datetime import datetime
import hashlib
from preprocessing_utils import ProbNNTransformer_fwd, ProbNNTransformer_bwd
from gpu_picker import request_gpu
print("Request gpu:", request_gpu(False))

2025-08-25 06:17:44.244658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-25 06:17:44.458658: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-25 06:17:44.461189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-25 06:17:44.867893: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-25 06:17:46.699782: W tensorflow/compiler/tf

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os, os.path
from os import environ

## Remove annoying warnings 
environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

In [3]:
from deploy_utils import LamarrModel, hacks

## Definition of a custom layer converter in scikinC
import scikinC.layers
scikinC.layers.DenseWithSkipConnection = hacks.scikinC_DenseWithSkipConnection

In [4]:
from IPython.display import Markdown
with open("pipeline.c") as pipeline:
    display(Markdown(f"``` c \n{pipeline.read()}\n```"))
    

``` c 
#define FLOAT_T float
// #define DEBUG 1

#ifndef BUFFERSIZE
#define BUFFERSIZE 1024
#endif 

#ifndef N_INPUT_RICH
#define N_INPUT_RICH 4
#endif 

#ifndef N_INPUT_MUON
#define N_INPUT_MUON 4
#endif 

#ifndef N_INPUT_GLOBALPID_MU  // MU: isMuon is True
#define N_INPUT_GLOBALPID_MU 10
#endif

#ifndef N_INPUT_GLOBALPID_HAD  // HAD: isMuon is False
#define N_INPUT_GLOBALPID_HAD 8
#endif 

#ifndef N_OUTPUT_RICH
#define N_OUTPUT_RICH 4
#endif 

#ifndef N_OUTPUT_MUON
#define N_OUTPUT_MUON 2
#endif 

#ifndef N_OUTPUT_GLOBALPID_MU
#define N_OUTPUT_GLOBALPID_MU 9
#endif 

#ifndef N_OUTPUT_GLOBALPID_HAD
#define N_OUTPUT_GLOBALPID_HAD 8
#endif 

#ifndef N_RANDOM_RICH
#define N_RANDOM_RICH 64
#endif 

#ifndef N_RANDOM_MUON
#define N_RANDOM_MUON 64
#endif 

#ifndef N_RANDOM_GLOBALPID_MU
#define N_RANDOM_GLOBALPID_MU 64
#endif 

#ifndef N_RANDOM_GLOBALPID_HAD
#define N_RANDOM_GLOBALPID_HAD 64
#endif 

#ifndef N_OUTPUT
#define N_OUTPUT 15
#endif 

#ifndef MUONLL_ERRORCODE
#define MUONLL_ERRORCODE -1000
#endif

#ifndef PROBMU_ERRORCODE
#define PROBMU_ERRORCODE -1
#endif

#ifdef DEBUG
#include <stdlib.h>
#include <stdio.h>
#endif 

FLOAT_T* Rich_muon_tX                    (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Rich_muon_tY_inverse            (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Rich_muon_dnn                   (FLOAT_T *, const FLOAT_T *);

FLOAT_T* Rich_pion_tX                    (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Rich_pion_tY_inverse            (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Rich_pion_dnn                   (FLOAT_T *, const FLOAT_T *);

FLOAT_T* Rich_kaon_tX                    (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Rich_kaon_tY_inverse            (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Rich_kaon_dnn                   (FLOAT_T *, const FLOAT_T *);

FLOAT_T* Rich_proton_tX                  (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Rich_proton_tY_inverse          (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Rich_proton_dnn                 (FLOAT_T *, const FLOAT_T *);

FLOAT_T* Muon_muon_tX                    (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Muon_muon_tY_inverse            (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Muon_muon_dnn                   (FLOAT_T *, const FLOAT_T *);

FLOAT_T* Muon_pion_tX                    (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Muon_pion_tY_inverse            (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Muon_pion_dnn                   (FLOAT_T *, const FLOAT_T *);

FLOAT_T* Muon_kaon_tX                    (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Muon_kaon_tY_inverse            (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Muon_kaon_dnn                   (FLOAT_T *, const FLOAT_T *);

FLOAT_T* Muon_proton_tX                  (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Muon_proton_tY_inverse          (FLOAT_T *, const FLOAT_T *);
FLOAT_T* Muon_proton_dnn                 (FLOAT_T *, const FLOAT_T *);

FLOAT_T* GlobalPIDmu_muon_tX             (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDmu_muon_tY_inverse     (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDmu_muon_dnn            (FLOAT_T *, const FLOAT_T *);

FLOAT_T* GlobalPIDmu_pion_tX             (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDmu_pion_tY_inverse     (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDmu_pion_dnn            (FLOAT_T *, const FLOAT_T *);

FLOAT_T* GlobalPIDmu_kaon_tX             (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDmu_kaon_tY_inverse     (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDmu_kaon_dnn            (FLOAT_T *, const FLOAT_T *);

FLOAT_T* GlobalPIDmu_proton_tX           (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDmu_proton_tY_inverse   (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDmu_proton_dnn          (FLOAT_T *, const FLOAT_T *);

FLOAT_T* GlobalPIDh_muon_tX              (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDh_muon_tY_inverse      (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDh_muon_dnn             (FLOAT_T *, const FLOAT_T *);

FLOAT_T* GlobalPIDh_pion_tX              (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDh_pion_tY_inverse      (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDh_pion_dnn             (FLOAT_T *, const FLOAT_T *);

FLOAT_T* GlobalPIDh_kaon_tX              (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDh_kaon_tY_inverse      (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDh_kaon_dnn             (FLOAT_T *, const FLOAT_T *);

FLOAT_T* GlobalPIDh_proton_tX            (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDh_proton_tY_inverse    (FLOAT_T *, const FLOAT_T *);
FLOAT_T* GlobalPIDh_proton_dnn           (FLOAT_T *, const FLOAT_T *);

typedef FLOAT_T* (*mlfun) (FLOAT_T *, const FLOAT_T *); 


FLOAT_T* gan_pipe ( mlfun tX               , 
                    mlfun model            , 
                    mlfun tY               , 
                    FLOAT_T *output        , 
                    const FLOAT_T *input   , 
                    const FLOAT_T *random  , 
                    unsigned short nIn     , 
                    unsigned short nOut    , 
                    unsigned short nRandom )
{
    unsigned short i; 
    FLOAT_T buf_input[BUFFERSIZE]; 
    FLOAT_T buf_output[BUFFERSIZE]; 

    tX (buf_input, input);

    #ifdef DEBUG
    printf("Preprocessed input\n");
    for (i = 0; i < nIn; ++i)
        printf("in[%d] -> pin[%d] : %.2f -> %.2f\n", i, i, input[i], buf_input[i]); 
    #endif

    for (i = 0; i < nRandom; ++i)
        buf_input[nIn + i] = random[i]; 

    model (buf_output, buf_input);

    tY (output, buf_output); 

    #ifdef DEBUG
    printf("Preprocessed output\n");
    for (i = 0; i < nOut; ++i)
        printf("pout[%d] -> out[%d] : %.2f -> %.2f\n", i, i, buf_output[i], output[i]); 
    #else
    (void)(nOut); // Suppress unused nOut warning
    #endif 

    return output; 
}


FLOAT_T* GenericPipe ( FLOAT_T* output , const FLOAT_T *input , const FLOAT_T *random ,
                       mlfun richTx   , mlfun richModel   , mlfun richTy   , 
                       mlfun muonTx   , mlfun muonModel   , mlfun muonTy   , 
                       mlfun gpidmuTx , mlfun gpidmuModel , mlfun gpidmuTy ,
                       mlfun gpidhTx  , mlfun gpidhModel  , mlfun gpidhTy  )
{
    short i, j;
    FLOAT_T isMuon;

    // Split the random array into four sub arrays
    j = 0; 
    const FLOAT_T* r0 = random + j;
    j += N_RANDOM_RICH; 
    const FLOAT_T* r1 = random + j;
    j += N_RANDOM_MUON; 
    const FLOAT_T* r2 = random + j;
    j += N_RANDOM_GLOBALPID_MU; 
    const FLOAT_T* r3 = random + j;

    // Rich
    FLOAT_T richinput [N_INPUT_RICH];
    FLOAT_T richdll [N_OUTPUT_RICH];
    for (i = 0; i < N_INPUT_RICH; ++i)
        richinput[i] = input[i]; 

    #ifdef DEBUG
    printf (" === RICH === \n");
    for (i = 0; i < N_INPUT_RICH; ++i)
        printf("RICH INPUT TRANSFER [%d] : %.2f -> %.2f\n", i, input[i], richinput[i]); 
    #endif

    gan_pipe ( richTx        ,
               richModel     ,
               richTy        ,
               richdll       ,
               richinput     ,
               r0            ,
               N_INPUT_RICH  ,
               N_OUTPUT_RICH ,
               N_RANDOM_RICH );

    isMuon = input[N_INPUT_RICH];

    if (isMuon > 0.5){

        // Muon
        FLOAT_T muoninput [N_INPUT_MUON];
        FLOAT_T muondll [N_OUTPUT_MUON];
        for (i = 0; i < N_INPUT_MUON; ++i)
            muoninput[i] = input[i]; 
    
        #ifdef DEBUG
        printf (" === MUON === \n");
        #endif

        gan_pipe ( muonTx        ,
                   muonModel     ,
                   muonTy        ,
                   muondll       ,
                   muoninput     ,
                   r1            ,
                   N_INPUT_RICH  ,
                   N_OUTPUT_RICH ,
                   N_RANDOM_RICH );

        // Global PID (muon)
        FLOAT_T gpidmu_input [N_INPUT_GLOBALPID_MU];
        FLOAT_T gpidmu_output [N_OUTPUT_GLOBALPID_MU];
        j = 0;

        // p, eta, nTracks
        for (i = 0; i < N_INPUT_RICH - 1; ++i)
            gpidmu_input[j++] = input[i]; 

        // dlle, dllmu, dllk, dllp
        for (i = 0; i < N_OUTPUT_RICH; ++i)
            gpidmu_input[j++] = richdll[i];

        // mullmu, mullbg
        for (i = 0; i < N_OUTPUT_MUON; ++i)
            gpidmu_input[j++] = muondll[i]; 

        // charge
        gpidmu_input[j++] = input[N_INPUT_RICH]; 

        #ifdef DEBUG
        printf (" === GLOBAL PID MUON === \n");
        #endif

        gan_pipe ( gpidmuTx              ,
                   gpidmuModel           ,
                   gpidmuTy              ,
                   gpidmu_output         ,
                   gpidmu_input          ,
                   r2                    ,
                   N_INPUT_GLOBALPID_MU  ,
                   N_OUTPUT_GLOBALPID_MU ,
                   N_RANDOM_GLOBALPID_MU ); 
    
        // Format output 
        j = 0;
        for (i = 0; i < N_OUTPUT_RICH; ++i)
            output[j++] = richdll[i]; 

        for (i = 0; i < N_OUTPUT_MUON; ++i)
            output[j++] = muondll[i]; 

        for (i = 0; i < N_OUTPUT_GLOBALPID_MU; ++i)
            output[j++] = gpidmu_output[i];

    }
    else {

        // Global PID (hadron)
        FLOAT_T gpidh_input [N_INPUT_GLOBALPID_HAD];
        FLOAT_T gpidh_output [N_OUTPUT_GLOBALPID_HAD];
        j = 0;

        // p, eta, nTracks
        for (i = 0; i < N_INPUT_RICH - 1; ++i)
            gpidh_input[j++] = input[i]; 

        // dlle, dllmu, dllk, dllp
        for (i = 0; i < N_OUTPUT_RICH; ++i)
            gpidh_input[j++] = richdll[i];

        // charge
        gpidh_input[j++] = input[N_INPUT_RICH]; 

        #ifdef DEBUG
        printf (" === GLOBAL PID HADRON === \n");
        #endif

        gan_pipe ( gpidhTx              ,
                   gpidhModel           ,
                   gpidhTy              ,
                   gpidh_output         ,
                   gpidh_input          ,
                   r3                    ,
                   N_INPUT_GLOBALPID_HAD  ,
                   N_OUTPUT_GLOBALPID_HAD ,
                   N_RANDOM_GLOBALPID_HAD ); 
    
        // Format output 
        j = 0;
        for (i = 0; i < N_OUTPUT_RICH; ++i)
            output[j++] = richdll[i]; 

        for (i = 0; i < N_OUTPUT_MUON; ++i)
            output[j++] = MUONLL_ERRORCODE;

        for (i = 0; i < N_OUTPUT_GLOBALPID_HAD; ++i)
            output[j++] = gpidh_output[i];

        for (i = N_OUTPUT_GLOBALPID_HAD; i < N_OUTPUT_GLOBALPID_MU; ++i)
            output[j++] = PROBMU_ERRORCODE;

    }

    #ifdef DEBUG
    printf (" === OUTPUT === \n");
    for (i = 0; i < N_OUTPUT; ++i)
        printf ( "in [%d] : %.2f\n", i, output[i] );
    #endif

    return output; 
}


FLOAT_T* full_muon_pipe (FLOAT_T* output, const FLOAT_T *input, const FLOAT_T *random)
{
    #ifdef DEBUG
    int i = 0;
    printf ("Muon pipe: INPUT\n") ;
    for (i = 0; i < N_INPUT_RICH; ++i)
        printf ( "in [%d] : %.2f\n", i, input[i]);

    printf ("Muon pipe: RANDOM\n") ;
    for (i = 0; i < N_RANDOM_RICH; ++i)
        printf ( "rnd [%d] : %.2f\n", i, random[i]);
    #endif 

    return GenericPipe ( output , input , random ,
                         Rich_muon_tX        , Rich_muon_dnn        , Rich_muon_tY_inverse        ,
                         Muon_muon_tX        , Muon_muon_dnn        , Muon_muon_tY_inverse        ,
                         GlobalPIDmu_muon_tX , GlobalPIDmu_muon_dnn , GlobalPIDmu_muon_tY_inverse ,
                         GlobalPIDh_muon_tX  , GlobalPIDh_muon_dnn  , GlobalPIDh_muon_tY_inverse  ); 
}


FLOAT_T* full_pion_pipe (FLOAT_T* output, const FLOAT_T *input, const FLOAT_T *random)
{
    #ifdef DEBUG
    int i = 0;
    printf ("Pion pipe: INPUT\n") ;
    for (i = 0; i < N_INPUT_RICH; ++i)
        printf ( "in [%d] : %.2f\n", i, input[i]);

    printf ("Pion pipe: RANDOM\n") ;
    for (i = 0; i < N_RANDOM_RICH; ++i)
        printf ( "rnd [%d] : %.2f\n", i, random[i]);
    #endif 

    return GenericPipe ( output , input , random ,
                         Rich_pion_tX        , Rich_pion_dnn        , Rich_pion_tY_inverse        ,
                         Muon_pion_tX        , Muon_pion_dnn        , Muon_pion_tY_inverse        ,
                         GlobalPIDmu_pion_tX , GlobalPIDmu_pion_dnn , GlobalPIDmu_pion_tY_inverse ,
                         GlobalPIDh_pion_tX  , GlobalPIDh_pion_dnn  , GlobalPIDh_pion_tY_inverse  ); 
}


FLOAT_T* full_kaon_pipe (FLOAT_T* output, const FLOAT_T *input, const FLOAT_T *random)
{
    #ifdef DEBUG
    int i = 0;
    printf ("Kaon pipe: INPUT\n") ;
    for (i = 0; i < N_INPUT_RICH; ++i)
        printf ( "in [%d] : %.2f\n", i, input[i]);

    printf ("Kaon pipe: RANDOM\n") ;
    for (i = 0; i < N_RANDOM_RICH; ++i)
        printf ( "rnd [%d] : %.2f\n", i, random[i]);
    #endif 

    return GenericPipe ( output , input , random ,
                         Rich_kaon_tX        , Rich_kaon_dnn        , Rich_kaon_tY_inverse        ,
                         Muon_kaon_tX        , Muon_kaon_dnn        , Muon_kaon_tY_inverse        ,
                         GlobalPIDmu_kaon_tX , GlobalPIDmu_kaon_dnn , GlobalPIDmu_kaon_tY_inverse ,
                         GlobalPIDh_kaon_tX  , GlobalPIDh_kaon_dnn  , GlobalPIDh_kaon_tY_inverse  ); 
}


FLOAT_T* full_proton_pipe (FLOAT_T* output, const FLOAT_T *input, const FLOAT_T *random)
{
    #ifdef DEBUG
    int i = 0;
    printf ("Proton pipe: INPUT\n") ;
    for (i = 0; i < N_INPUT_RICH; ++i)
        printf ( "in [%d] : %.2f\n", i, input[i]);

    printf ("Proton pipe: RANDOM\n") ;
    for (i = 0; i < N_RANDOM_RICH; ++i)
        printf ( "rnd [%d] : %.2f\n", i, random[i]);
    #endif 

    return GenericPipe ( output , input , random ,
                         Rich_proton_tX        , Rich_proton_dnn        , Rich_proton_tY_inverse        ,
                         Muon_proton_tX        , Muon_proton_dnn        , Muon_proton_tY_inverse        ,
                         GlobalPIDmu_proton_tX , GlobalPIDmu_proton_dnn , GlobalPIDmu_proton_tY_inverse ,
                         GlobalPIDh_proton_tX  , GlobalPIDh_proton_dnn  , GlobalPIDh_proton_tY_inverse  ); 
}

```

In [5]:
import os
from deploy_utils import LamarrModel

model_names = [
    'isMuon', 
    'Rich', 
    'Muon', 
    'GlobalPID-im', 
    'GlobalPID-nm',
] 
models_without_postprocessing = ['isMuon']
particles = ['muon', 'pion', 'kaon', 'proton']

def fn_name(model, particle, step):
    assert step in ['dnn', 'tX', 'tY', 'pipe'], f"Invalid step: {step}"
    name_replacements = {
        'GlobalPID-im': 'GlobalPIDmu', 
        'GlobalPID-nm': 'GlobalPIDh',
    }

    model = name_replacements.get(model, model)
    
    return f'{model}_{particle}_{step}'

models = dict()
for model in model_names:
    for particle in particles:
        env_prefix = f"{model.upper().replace('-', '_')}_{particle.upper()}"
        print(f"Processing {env_prefix}...")
        loaded_model = LamarrModel.from_saved_model_pb(
            os.environ.get(
                f"{env_prefix}_MODEL", 
                f"/tmp/s3models/lamarr-train/models/{model}/{particle}/model.keras"
            )
        )

        # hack for managing the change in the new naming convention adopted by keras
        for layer in loaded_model.collapsed_model.layers:
            if layer.__class__.__name__ == 'LeakyReLU':
                layer.alpha = layer.negative_slope

        if model in models_without_postprocessing:
            loaded_model._tY = None
            models[fn_name(model, particle, 'pipe')] = loaded_model.pipeline
        else:
            models[fn_name(model, particle, 'dnn')] = loaded_model.collapsed_model
            models[fn_name(model, particle, 'tX')] = loaded_model.tX
            models[fn_name(model, particle, 'tY')] = loaded_model.tY
                

Processing ISMUON_MUON...
Loading model from '/tmp/s3models/lamarr-train/models/isMuon/muon'.  Preprocessing: 👌.Postprocessing: 👌.


2025-08-25 06:17:51.120927: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-25 06:17:51.695901: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-25 06:17:51.697462: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Check on the number of weights: ✅! Original model: 149377. Collapsed model: 149377. 


/usr/local/lib/python3.12/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_231
Received: inputs=('Tensor(shape=(1, 4))',)
  warnings.warn(msg)
I0000 00:00:1756102673.861655  330562 service.cc:145] XLA service 0x7f55d8003500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756102673.861734  330562 service.cc:153]   StreamExecutor device (0): NVIDIA A100 80GB PCIe MIG 1g.10gb, Compute Capability 8.0
2025-08-25 06:17:53.885603: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-25 06:17:53.987500: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1756102674.102647  330562 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of t

Processing ISMUON_PION...
Loading model from '/tmp/s3models/lamarr-train/models/isMuon/pion'.  Preprocessing: 👌.Postprocessing: 👌.
Check on the number of weights: ✅! Original model: 149377. Collapsed model: 149377. 


...

/usr/local/lib/python3.12/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_1859
Received: inputs=('Tensor(shape=(1, 72))',)
  warnings.warn(msg)


Processing GLOBALPID_NM_KAON...
Loading model from '/tmp/s3models/lamarr-train/models/GlobalPID-nm/kaon'.  Preprocessing: 👌.Postprocessing: 👌.
Check on the number of weights: ✅! Original model: 76424. Collapsed model: 76424. 


/usr/local/lib/python3.12/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_1900
Received: inputs=('Tensor(shape=(1, 72))',)
  warnings.warn(msg)


Processing GLOBALPID_NM_PROTON...
Loading model from '/tmp/s3models/lamarr-train/models/GlobalPID-nm/proton'.  Preprocessing: 👌.Postprocessing: 👌.
Check on the number of weights: ✅! Original model: 76424. Collapsed model: 76424. 


/usr/local/lib/python3.12/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_1941
Received: inputs=('Tensor(shape=(1, 72))',)
  warnings.warn(msg)


In [6]:
# Retrieve the name of the file from the environment
generated_c = environ.get("GENERATED_C_FILE", "/tmp/exported/generated.c")

os.makedirs(os.path.dirname(generated_c), exist_ok=True)

with open(generated_c, 'w') as output_file:
    # Runs the scikinC code generator and save the output in the generated_c file
    print(f'#pragma message "Using Lamarr PID models as exported on {datetime.now()}"', file=output_file)
    print(open('pipeline.c').read(), file=output_file)
    print(scikinC.convert({k: v for k, v in models.items() if 'GlobalPIDmu_muon' in k}).replace('extern "C"', ''), file=output_file)


In [7]:
# Retrieve the name of the file from the environment
generated_c = environ.get("GENERATED_C_FILE", "/tmp/exported/generated.c")

os.makedirs(os.path.dirname(generated_c), exist_ok=True)

with open(generated_c, 'w') as output_file:
    # Runs the scikinC code generator and save the output in the generated_c file
    print(f'#pragma message "Using Lamarr PID models as exported on {datetime.now()}"', file=output_file)
    print(open('pipeline.c').read(), file=output_file)
    print(scikinC.convert(models).replace('extern "C"', ''), file=output_file)


In [8]:
import time, subprocess
default_lib_path = generated_c.replace('.c', str(time.time()) + '.so')
lib_path = environ.get("GENERATED_LIBRARY", default_lib_path)


cmd = [
    "gcc",
    generated_c,
    f"-o{lib_path}",
    # "-O3", "-flto", "-fno-fast-math", "-fno-math-errno",
    "-lm", "--shared", "-fPIC", "-std=c99", "-Wall", "-Wextra", "-pedantic-errors",
]
print ("Compilation command:\n", "  \\\n  ".join(cmd))
#os.system(" ".join(cmd))

Compilation command:
 gcc  \
  /tmp/exported/generated.c  \
  -o/tmp/exported/generated1756102685.532006.so  \
  -lm  \
  --shared  \
  -fPIC  \
  -std=c99  \
  -Wall  \
  -Wextra  \
  -pedantic-errors


In [9]:
import subprocess 

result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

linking_symbols = subprocess.run(['nm', '-D', lib_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [10]:
symbols = [s.split(' ')[-1] for s in str(linking_symbols.stdout).split('\\n')]
symbols = '\n'.join(
    [' '.join([f"{s:68s}" for s in row]) for row in zip(symbols[0::3], symbols[1::3], symbols[2::3])]
)

In [11]:
display(Markdown(f"""
## Stats
 * Generated C file size: `{os.path.getsize(generated_c) / 2**20:.1f} MiB`
 * MD5: `{hashlib.md5(open(generated_c, 'rb').read()).hexdigest()}`


## Compiler command
---
``` bash \n{' \\\n  '.join(cmd)}\n```  
---

## GCC output 

---
``` c \n{str(result.stderr[:100000], 'utf-8')} \n```
---

## Linking Symbols 
``` c \n{symbols} \n```

"""))


## Stats
 * Generated C file size: `71.1 MiB`
 * MD5: `90047af4f4d4bf02bf7ec8bb62931686`


## Compiler command
---
``` bash 
gcc \
  /tmp/exported/generated.c \
  -o/tmp/exported/generated1756102685.532006.so \
  -lm \
  --shared \
  -fPIC \
  -std=c99 \
  -Wall \
  -Wextra \
  -pedantic-errors
```  
---

## GCC output 

---
``` c 
/tmp/exported/generated.c:1:9: note: ‘#pragma message: Using Lamarr PID models as exported on 2025-08-25 06:18:01.608275’
    1 | #pragma message "Using Lamarr PID models as exported on 2025-08-25 06:18:01.608275"
      |         ^~~~~~~
 
```
---

## Linking Symbols 
``` c 
GenericPipe                                                          GlobalPIDh_kaon_dnn                                                  GlobalPIDh_kaon_dnn_cpy_dense_0                                     
GlobalPIDh_kaon_dnn_cpy_dense_out                                    GlobalPIDh_kaon_dnn_resdense_1                                       GlobalPIDh_kaon_dnn_resdense_2                                      
GlobalPIDh_kaon_dnn_resdense_3                                       GlobalPIDh_kaon_dnn_resdense_4                                       GlobalPIDh_kaon_tX                                                  
GlobalPIDh_kaon_tX_inverse                                           GlobalPIDh_kaon_tY                                                   GlobalPIDh_kaon_tY_inverse                                          
GlobalPIDh_muon_dnn                                                  GlobalPIDh_muon_dnn_cpy_dense_0                                      GlobalPIDh_muon_dnn_cpy_dense_out                                   
GlobalPIDh_muon_dnn_resdense_1                                       GlobalPIDh_muon_dnn_resdense_2                                       GlobalPIDh_muon_dnn_resdense_3                                      
GlobalPIDh_muon_dnn_resdense_4                                       GlobalPIDh_muon_tX                                                   GlobalPIDh_muon_tX_inverse                                          
GlobalPIDh_muon_tY                                                   GlobalPIDh_muon_tY_inverse                                           GlobalPIDh_pion_dnn                                                 
GlobalPIDh_pion_dnn_cpy_dense_0                                      GlobalPIDh_pion_dnn_cpy_dense_out                                    GlobalPIDh_pion_dnn_resdense_1                                      
GlobalPIDh_pion_dnn_resdense_2                                       GlobalPIDh_pion_dnn_resdense_3                                       GlobalPIDh_pion_dnn_resdense_4                                      
GlobalPIDh_pion_tX                                                   GlobalPIDh_pion_tX_inverse                                           GlobalPIDh_pion_tY                                                  
GlobalPIDh_pion_tY_inverse                                           GlobalPIDh_proton_dnn                                                GlobalPIDh_proton_dnn_cpy_dense_0                                   
GlobalPIDh_proton_dnn_cpy_dense_out                                  GlobalPIDh_proton_dnn_resdense_1                                     GlobalPIDh_proton_dnn_resdense_2                                    
GlobalPIDh_proton_dnn_resdense_3                                     GlobalPIDh_proton_dnn_resdense_4                                     GlobalPIDh_proton_tX                                                
GlobalPIDh_proton_tX_inverse                                         GlobalPIDh_proton_tY                                                 GlobalPIDh_proton_tY_inverse                                        
GlobalPIDmu_kaon_dnn                                                 GlobalPIDmu_kaon_dnn_cpy_dense_0                                     GlobalPIDmu_kaon_dnn_cpy_dense_out                                  
GlobalPIDmu_kaon_dnn_resdense_1                                      GlobalPIDmu_kaon_dnn_resdense_2                                      GlobalPIDmu_kaon_dnn_resdense_3                                     
GlobalPIDmu_kaon_dnn_resdense_4                                      GlobalPIDmu_kaon_tX                                                  GlobalPIDmu_kaon_tX_inverse                                         
GlobalPIDmu_kaon_tY                                                  GlobalPIDmu_kaon_tY_inverse                                          GlobalPIDmu_muon_dnn                                                
GlobalPIDmu_muon_dnn_cpy_dense_0                                     GlobalPIDmu_muon_dnn_cpy_dense_out                                   GlobalPIDmu_muon_dnn_resdense_1                                     
GlobalPIDmu_muon_dnn_resdense_2                                      GlobalPIDmu_muon_dnn_resdense_3                                      GlobalPIDmu_muon_dnn_resdense_4                                     
GlobalPIDmu_muon_tX                                                  GlobalPIDmu_muon_tX_inverse                                          GlobalPIDmu_muon_tY                                                 
GlobalPIDmu_muon_tY_inverse                                          GlobalPIDmu_pion_dnn                                                 GlobalPIDmu_pion_dnn_cpy_dense_0                                    
GlobalPIDmu_pion_dnn_cpy_dense_out                                   GlobalPIDmu_pion_dnn_resdense_1                                      GlobalPIDmu_pion_dnn_resdense_2                                     
GlobalPIDmu_pion_dnn_resdense_3                                      GlobalPIDmu_pion_dnn_resdense_4                                      GlobalPIDmu_pion_tX                                                 
GlobalPIDmu_pion_tX_inverse                                          GlobalPIDmu_pion_tY                                                  GlobalPIDmu_pion_tY_inverse                                         
GlobalPIDmu_proton_dnn                                               GlobalPIDmu_proton_dnn_cpy_dense_0                                   GlobalPIDmu_proton_dnn_cpy_dense_out                                
GlobalPIDmu_proton_dnn_resdense_1                                    GlobalPIDmu_proton_dnn_resdense_2                                    GlobalPIDmu_proton_dnn_resdense_3                                   
GlobalPIDmu_proton_dnn_resdense_4                                    GlobalPIDmu_proton_tX                                                GlobalPIDmu_proton_tX_inverse                                       
GlobalPIDmu_proton_tY                                                GlobalPIDmu_proton_tY_inverse                                        Muon_kaon_dnn                                                       
Muon_kaon_dnn_cpy_dense_0                                            Muon_kaon_dnn_cpy_dense_out                                          Muon_kaon_dnn_resdense_1                                            
Muon_kaon_dnn_resdense_2                                             Muon_kaon_dnn_resdense_3                                             Muon_kaon_dnn_resdense_4                                            
Muon_kaon_dnn_resdense_5                                             Muon_kaon_dnn_resdense_6                                             Muon_kaon_dnn_resdense_7                                            
Muon_kaon_dnn_resdense_8                                             Muon_kaon_dnn_resdense_9                                             Muon_kaon_tX                                                        
Muon_kaon_tX_inverse                                                 Muon_kaon_tY                                                         Muon_kaon_tY_inverse                                                
Muon_muon_dnn                                                        Muon_muon_dnn_cpy_dense_0                                            Muon_muon_dnn_cpy_dense_out                                         
Muon_muon_dnn_resdense_1                                             Muon_muon_dnn_resdense_2                                             Muon_muon_dnn_resdense_3                                            
Muon_muon_dnn_resdense_4                                             Muon_muon_dnn_resdense_5                                             Muon_muon_dnn_resdense_6                                            
Muon_muon_dnn_resdense_7                                             Muon_muon_dnn_resdense_8                                             Muon_muon_dnn_resdense_9                                            
Muon_muon_tX                                                         Muon_muon_tX_inverse                                                 Muon_muon_tY                                                        
Muon_muon_tY_inverse                                                 Muon_pion_dnn                                                        Muon_pion_dnn_cpy_dense_0                                           
Muon_pion_dnn_cpy_dense_out                                          Muon_pion_dnn_resdense_1                                             Muon_pion_dnn_resdense_2                                            
Muon_pion_dnn_resdense_3                                             Muon_pion_dnn_resdense_4                                             Muon_pion_dnn_resdense_5                                            
Muon_pion_dnn_resdense_6                                             Muon_pion_dnn_resdense_7                                             Muon_pion_dnn_resdense_8                                            
Muon_pion_dnn_resdense_9                                             Muon_pion_tX                                                         Muon_pion_tX_inverse                                                
Muon_pion_tY                                                         Muon_pion_tY_inverse                                                 Muon_proton_dnn                                                     
Muon_proton_dnn_cpy_dense_0                                          Muon_proton_dnn_cpy_dense_out                                        Muon_proton_dnn_resdense_1                                          
Muon_proton_dnn_resdense_2                                           Muon_proton_dnn_resdense_3                                           Muon_proton_dnn_resdense_4                                          
Muon_proton_dnn_resdense_5                                           Muon_proton_dnn_resdense_6                                           Muon_proton_dnn_resdense_7                                          
Muon_proton_dnn_resdense_8                                           Muon_proton_dnn_resdense_9                                           Muon_proton_tX                                                      
Muon_proton_tX_inverse                                               Muon_proton_tY                                                       Muon_proton_tY_inverse                                              
Rich_kaon_dnn                                                        Rich_kaon_dnn_cpy_dense_0                                            Rich_kaon_dnn_cpy_dense_out                                         
Rich_kaon_dnn_resdense_1                                             Rich_kaon_dnn_resdense_2                                             Rich_kaon_dnn_resdense_3                                            
Rich_kaon_dnn_resdense_4                                             Rich_kaon_dnn_resdense_5                                             Rich_kaon_dnn_resdense_6                                            
Rich_kaon_dnn_resdense_7                                             Rich_kaon_dnn_resdense_8                                             Rich_kaon_dnn_resdense_9                                            
Rich_kaon_tX                                                         Rich_kaon_tX_inverse                                                 Rich_kaon_tY                                                        
Rich_kaon_tY_inverse                                                 Rich_muon_dnn                                                        Rich_muon_dnn_cpy_dense_0                                           
Rich_muon_dnn_cpy_dense_out                                          Rich_muon_dnn_resdense_1                                             Rich_muon_dnn_resdense_2                                            
Rich_muon_dnn_resdense_3                                             Rich_muon_dnn_resdense_4                                             Rich_muon_dnn_resdense_5                                            
Rich_muon_dnn_resdense_6                                             Rich_muon_dnn_resdense_7                                             Rich_muon_dnn_resdense_8                                            
Rich_muon_dnn_resdense_9                                             Rich_muon_tX                                                         Rich_muon_tX_inverse                                                
Rich_muon_tY                                                         Rich_muon_tY_inverse                                                 Rich_pion_dnn                                                       
Rich_pion_dnn_cpy_dense_0                                            Rich_pion_dnn_cpy_dense_out                                          Rich_pion_dnn_resdense_1                                            
Rich_pion_dnn_resdense_2                                             Rich_pion_dnn_resdense_3                                             Rich_pion_dnn_resdense_4                                            
Rich_pion_dnn_resdense_5                                             Rich_pion_dnn_resdense_6                                             Rich_pion_dnn_resdense_7                                            
Rich_pion_dnn_resdense_8                                             Rich_pion_dnn_resdense_9                                             Rich_pion_tX                                                        
Rich_pion_tX_inverse                                                 Rich_pion_tY                                                         Rich_pion_tY_inverse                                                
Rich_proton_dnn                                                      Rich_proton_dnn_cpy_dense_0                                          Rich_proton_dnn_cpy_dense_out                                       
Rich_proton_dnn_resdense_1                                           Rich_proton_dnn_resdense_2                                           Rich_proton_dnn_resdense_3                                          
Rich_proton_dnn_resdense_4                                           Rich_proton_dnn_resdense_5                                           Rich_proton_dnn_resdense_6                                          
Rich_proton_dnn_resdense_7                                           Rich_proton_dnn_resdense_8                                           Rich_proton_dnn_resdense_9                                          
Rich_proton_tX                                                       Rich_proton_tX_inverse                                               Rich_proton_tY                                                      
Rich_proton_tY_inverse                                               _ITM_deregisterTMCloneTable                                          _ITM_registerTMCloneTable                                           
__cxa_finalize@GLIBC_2.2.5                                           __gmon_start__                                                       colcnv_GlobalPIDh_kaon_tX_quantile                                  
colcnv_GlobalPIDh_kaon_tX_quantile_inverse                           colcnv_GlobalPIDh_kaon_tY_probs                                      colcnv_GlobalPIDh_kaon_tY_probs_inverse                             
colcnv_GlobalPIDh_kaon_tY_standard                                   colcnv_GlobalPIDh_kaon_tY_standard_inverse                           colcnv_GlobalPIDh_muon_tX_quantile                                  
colcnv_GlobalPIDh_muon_tX_quantile_inverse                           colcnv_GlobalPIDh_muon_tY_probs                                      colcnv_GlobalPIDh_muon_tY_probs_inverse                             
colcnv_GlobalPIDh_muon_tY_standard                                   colcnv_GlobalPIDh_muon_tY_standard_inverse                           colcnv_GlobalPIDh_pion_tX_quantile                                  
colcnv_GlobalPIDh_pion_tX_quantile_inverse                           colcnv_GlobalPIDh_pion_tY_probs                                      colcnv_GlobalPIDh_pion_tY_probs_inverse                             
colcnv_GlobalPIDh_pion_tY_standard                                   colcnv_GlobalPIDh_pion_tY_standard_inverse                           colcnv_GlobalPIDh_proton_tX_quantile                                
colcnv_GlobalPIDh_proton_tX_quantile_inverse                         colcnv_GlobalPIDh_proton_tY_probs                                    colcnv_GlobalPIDh_proton_tY_probs_inverse                           
colcnv_GlobalPIDh_proton_tY_standard                                 colcnv_GlobalPIDh_proton_tY_standard_inverse                         colcnv_GlobalPIDmu_kaon_tX_quantile                                 
colcnv_GlobalPIDmu_kaon_tX_quantile_inverse                          colcnv_GlobalPIDmu_kaon_tX_standard                                  colcnv_GlobalPIDmu_kaon_tX_standard_inverse                         
colcnv_GlobalPIDmu_kaon_tY_probs                                     colcnv_GlobalPIDmu_kaon_tY_probs_inverse                             colcnv_GlobalPIDmu_kaon_tY_standard                                 
colcnv_GlobalPIDmu_kaon_tY_standard_inverse                          colcnv_GlobalPIDmu_muon_tX_quantile                                  colcnv_GlobalPIDmu_muon_tX_quantile_inverse                         
colcnv_GlobalPIDmu_muon_tX_standard                                  colcnv_GlobalPIDmu_muon_tX_standard_inverse                          colcnv_GlobalPIDmu_muon_tY_probs                                    
colcnv_GlobalPIDmu_muon_tY_probs_inverse                             colcnv_GlobalPIDmu_muon_tY_standard                                  colcnv_GlobalPIDmu_muon_tY_standard_inverse                         
colcnv_GlobalPIDmu_pion_tX_quantile                                  colcnv_GlobalPIDmu_pion_tX_quantile_inverse                          colcnv_GlobalPIDmu_pion_tX_standard                                 
colcnv_GlobalPIDmu_pion_tX_standard_inverse                          colcnv_GlobalPIDmu_pion_tY_probs                                     colcnv_GlobalPIDmu_pion_tY_probs_inverse                            
colcnv_GlobalPIDmu_pion_tY_standard                                  colcnv_GlobalPIDmu_pion_tY_standard_inverse                          colcnv_GlobalPIDmu_proton_tX_quantile                               
colcnv_GlobalPIDmu_proton_tX_quantile_inverse                        colcnv_GlobalPIDmu_proton_tX_standard                                colcnv_GlobalPIDmu_proton_tX_standard_inverse                       
colcnv_GlobalPIDmu_proton_tY_probs                                   colcnv_GlobalPIDmu_proton_tY_probs_inverse                           colcnv_GlobalPIDmu_proton_tY_standard                               
colcnv_GlobalPIDmu_proton_tY_standard_inverse                        colcnv_Muon_kaon_tX_features                                         colcnv_Muon_kaon_tX_features_inverse                                
colcnv_Muon_muon_tX_features                                         colcnv_Muon_muon_tX_features_inverse                                 colcnv_Muon_pion_tX_features                                        
colcnv_Muon_pion_tX_features_inverse                                 colcnv_Muon_proton_tX_features                                       colcnv_Muon_proton_tX_features_inverse                              
colcnv_Rich_kaon_tX_features                                         colcnv_Rich_kaon_tX_features_inverse                                 colcnv_Rich_muon_tX_features                                        
colcnv_Rich_muon_tX_features_inverse                                 colcnv_Rich_pion_tX_features                                         colcnv_Rich_pion_tX_features_inverse                                
colcnv_Rich_proton_tX_features                                       colcnv_Rich_proton_tX_features_inverse                               colcnv_isMuon_kaon_pipe_preprocessing_features                      
colcnv_isMuon_kaon_pipe_preprocessing_features_inverse               colcnv_isMuon_muon_pipe_preprocessing_features                       colcnv_isMuon_muon_pipe_preprocessing_features_inverse              
colcnv_isMuon_pion_pipe_preprocessing_features                       colcnv_isMuon_pion_pipe_preprocessing_features_inverse               colcnv_isMuon_proton_pipe_preprocessing_features                    
colcnv_isMuon_proton_pipe_preprocessing_features_inverse             exp@GLIBC_2.29                                                       full_kaon_pipe                                                      
full_muon_pipe                                                       full_pion_pipe                                                       full_proton_pipe                                                    
gan_pipe                                                             isMuon_kaon_pipe                                                     isMuon_kaon_pipe_dnn                                                
isMuon_kaon_pipe_dnn_cpy_dense_0                                     isMuon_kaon_pipe_dnn_cpy_dense_1                                     isMuon_kaon_pipe_dnn_cpy_dense_2                                    
isMuon_kaon_pipe_dnn_cpy_dense_3                                     isMuon_kaon_pipe_dnn_cpy_dense_4                                     isMuon_kaon_pipe_dnn_cpy_dense_5                                    
isMuon_kaon_pipe_dnn_cpy_dense_6                                     isMuon_kaon_pipe_dnn_cpy_dense_7                                     isMuon_kaon_pipe_dnn_cpy_dense_8                                    
isMuon_kaon_pipe_dnn_cpy_dense_9                                     isMuon_kaon_pipe_dnn_cpy_dense_out                                   isMuon_kaon_pipe_preprocessing                                      
isMuon_kaon_pipe_preprocessing_inverse                               isMuon_muon_pipe                                                     isMuon_muon_pipe_dnn                                                
isMuon_muon_pipe_dnn_cpy_dense_0                                     isMuon_muon_pipe_dnn_cpy_dense_1                                     isMuon_muon_pipe_dnn_cpy_dense_2                                    
isMuon_muon_pipe_dnn_cpy_dense_3                                     isMuon_muon_pipe_dnn_cpy_dense_4                                     isMuon_muon_pipe_dnn_cpy_dense_5                                    
isMuon_muon_pipe_dnn_cpy_dense_6                                     isMuon_muon_pipe_dnn_cpy_dense_7                                     isMuon_muon_pipe_dnn_cpy_dense_8                                    
isMuon_muon_pipe_dnn_cpy_dense_9                                     isMuon_muon_pipe_dnn_cpy_dense_out                                   isMuon_muon_pipe_preprocessing                                      
isMuon_muon_pipe_preprocessing_inverse                               isMuon_pion_pipe                                                     isMuon_pion_pipe_dnn                                                
isMuon_pion_pipe_dnn_cpy_dense_0                                     isMuon_pion_pipe_dnn_cpy_dense_1                                     isMuon_pion_pipe_dnn_cpy_dense_2                                    
isMuon_pion_pipe_dnn_cpy_dense_3                                     isMuon_pion_pipe_dnn_cpy_dense_4                                     isMuon_pion_pipe_dnn_cpy_dense_5                                    
isMuon_pion_pipe_dnn_cpy_dense_6                                     isMuon_pion_pipe_dnn_cpy_dense_7                                     isMuon_pion_pipe_dnn_cpy_dense_8                                    
isMuon_pion_pipe_dnn_cpy_dense_9                                     isMuon_pion_pipe_dnn_cpy_dense_out                                   isMuon_pion_pipe_preprocessing                                      
isMuon_pion_pipe_preprocessing_inverse                               isMuon_proton_pipe                                                   isMuon_proton_pipe_dnn                                              
isMuon_proton_pipe_dnn_cpy_dense_0                                   isMuon_proton_pipe_dnn_cpy_dense_1                                   isMuon_proton_pipe_dnn_cpy_dense_2                                  
isMuon_proton_pipe_dnn_cpy_dense_3                                   isMuon_proton_pipe_dnn_cpy_dense_4                                   isMuon_proton_pipe_dnn_cpy_dense_5                                  
isMuon_proton_pipe_dnn_cpy_dense_6                                   isMuon_proton_pipe_dnn_cpy_dense_7                                   isMuon_proton_pipe_dnn_cpy_dense_8                                  
isMuon_proton_pipe_dnn_cpy_dense_9                                   isMuon_proton_pipe_dnn_cpy_dense_out                                 isMuon_proton_pipe_preprocessing                                    
isMuon_proton_pipe_preprocessing_inverse                             log@GLIBC_2.29                                                       memcpy@GLIBC_2.14                                                   
qtc_interpolate_for_colcnv_GlobalPIDh_kaon_tX_quantile               qtc_interpolate_for_colcnv_GlobalPIDh_muon_tX_quantile               qtc_interpolate_for_colcnv_GlobalPIDh_pion_tX_quantile              
qtc_interpolate_for_colcnv_GlobalPIDh_proton_tX_quantile             qtc_interpolate_for_colcnv_GlobalPIDmu_kaon_tX_quantile              qtc_interpolate_for_colcnv_GlobalPIDmu_muon_tX_quantile             
qtc_interpolate_for_colcnv_GlobalPIDmu_pion_tX_quantile              qtc_interpolate_for_colcnv_GlobalPIDmu_proton_tX_quantile            qtc_interpolate_for_colcnv_Muon_kaon_tX_features                    
qtc_interpolate_for_colcnv_Muon_muon_tX_features                     qtc_interpolate_for_colcnv_Muon_pion_tX_features                     qtc_interpolate_for_colcnv_Muon_proton_tX_features                  
qtc_interpolate_for_colcnv_Rich_kaon_tX_features                     qtc_interpolate_for_colcnv_Rich_muon_tX_features                     qtc_interpolate_for_colcnv_Rich_pion_tX_features                    
qtc_interpolate_for_colcnv_Rich_proton_tX_features                   qtc_interpolate_for_colcnv_isMuon_kaon_pipe_preprocessing_features   qtc_interpolate_for_colcnv_isMuon_muon_pipe_preprocessing_features  
qtc_interpolate_for_colcnv_isMuon_pion_pipe_preprocessing_features   qtc_interpolate_for_colcnv_isMuon_proton_pipe_preprocessing_features '                                                                    
```

